In [48]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
data = pd.read_csv("tweets_lemm_train.csv")
corpus = data['lemm_text'].values.astype('U')

nltk.download('stopwords')
stopwords = list(set(nltk_stopwords.words('russian')))

count_tf_idf = TfidfVectorizer(stop_words = stopwords) 
tf_idf = count_tf_idf.fit_transform(corpus) 

print("Размер матрицы:", tf_idf.shape)

Размер матрицы: (5000, 9737)


[nltk_data] Downloading package stopwords to /home/ltz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [33]:
tf_idf[0,4500:5500].todense()

matrix([[0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0

In [37]:
y = data.positive
X = tf_idf

In [60]:
gs = GridSearchCV(LogisticRegression(random_state=4999),
                 {'max_iter':[2000],'class_weight':['balanced'],'solver':['newton-cg', 'lbfgs', 'liblinear']},
                  cv= 3)
gs.fit(X,y)
gs.cv_results_

{'mean_fit_time': array([0.02327236, 0.02752296, 0.00458527]),
 'std_fit_time': array([0.00641132, 0.00241789, 0.00059598]),
 'mean_score_time': array([0.00077264, 0.00084615, 0.00065215]),
 'std_score_time': array([0.00021073, 0.00010607, 0.00013764]),
 'param_class_weight': masked_array(data=['balanced', 'balanced', 'balanced'],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'param_max_iter': masked_array(data=[2000, 2000, 2000],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'param_solver': masked_array(data=['newton-cg', 'lbfgs', 'liblinear'],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'class_weight': 'balanced',
   'max_iter': 2000,
   'solver': 'newton-cg'},
  {'class_weight': 'balanced', 'max_iter': 2000, 'solver': 'lbfgs'},
  {'class_weight': 'balanced', 'max_iter': 2000, 'solver': 'liblinear'}],
 'split0_test_score': ar

In [38]:
lrm = LogisticRegression()
lrm.fit(X,y)

LogisticRegression()

In [39]:
test_data = pd.read_csv("tweets_lemm_test.csv")

In [43]:
X_te = count_tf_idf.transform(test_data.lemm_text.values.astype('U') )
X_te

<3000x9737 sparse matrix of type '<class 'numpy.float64'>'
	with 15112 stored elements in Compressed Sparse Row format>

In [63]:
pr = pd.Series( lrm.predict(X_te), name= 'positive', index= test_data.index )

In [67]:
pd.concat([ test_data, pr],axis= 1).to_csv( 'predictions' ,index=False)